# Exploring OpenAI V1 functionality

On 11.06.23 OpenAI released a number of new features, and along with it bumped their Python SDK to 1.0.0. This notebook shows off the new features and how to use them with LangChain.

In [ ]:
!pip install "openai>=1"

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage

## [Vision](https://platform.openai.com/docs/guides/vision)

OpenAI released multi-modal models, which can take a sequence of text and images as input.

In [2]:
chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=256)
chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "What is this image showing"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://python.langchain.com/assets/images/langchain_stack-da369071b058555da3d491a695651f15.jpg",
                        "detail": "auto",
                    },
                },
            ]
        )
    ]
)

AIMessage(content='The image appears to be a diagram representing the architecture or components of a software platform named "LangChain." This diagram outlines various layers and elements of the platform, which seems to be related to language processing or computational linguistics, as suggested by the context clues in the names of the components.\n\nHere\'s a breakdown of the components shown:\n\n- **LangSmith**: This seems to be a tool or suite related to testing, evaluation, monitoring, feedback, and annotation within the platform.\n\n- **LangServe**: This could represent a service layer that exposes the platform\'s capabilities as REST API endpoints.\n\n- **Templates**: These are likely reference applications provided as starting points or examples for users of the platform.\n\n- **Chains, agents, agent executors**: This section describes the common application logic, perhaps indicating that the platform uses a chain of agents or processes to execute tasks.\n\n- **Model I/O**: Thi

## [OpenAI assistants](https://platform.openai.com/docs/assistants/overview)

> The Assistants API allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, Retrieval, and Function calling

In [6]:
from langchain_experimental.openai_assistant.base import OpenAIAssistantRunnable

interpreter_assistant = OpenAIAssistantRunnable(
    name="langchain assistant",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)
interpreter_assistant.create()
output = interpreter_assistant.invoke({"content": "What's 10 - 4 raised to the 2.7"})
output

ValueError: {'id': 'run_PiaUaLysMmWCgT49jLNkBhhx', 'assistant_id': 'asst_4mXntqWdxkAM8CLkGERlYBXI', 'cancelled_at': None, 'completed_at': None, 'created_at': 1699337403, 'expires_at': None, 'failed_at': 1699337413, 'file_ids': [], 'instructions': 'You are a personal math tutor. Write and run code to answer math questions.', 'last_error': {'code': 'rate_limit_exceeded', 'message': "We're currently processing too many requests - please try again later."}, 'metadata': {}, 'model': 'gpt-4-1106-preview', 'object': 'thread.run', 'required_action': None, 'started_at': 1699337403, 'status': 'failed', 'thread_id': 'thread_mRVqIj5Jm3rxuGreucH22bsr', 'tools': [{'type': 'code_interpreter'}]}

### As a LangChain agent with arbitrary tools

In [7]:
from langchain.tools import E2BDataAnalysisTool, format_tool_to_openai_function

tool = E2BDataAnalysisTool(api_key="...")
function = format_tool_to_openai_function(tool)
tools = [tool]

Failed to acquire session


UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 07 Nov 2023 06:10:27 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '123', 'Connection': 'keep-alive', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=86400', 'CF-Cache-Status': 'DYNAMIC', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v3?s=eD4xvpIGOuei6fNw3tUox8hcw2rilvLB%2FUrFxgAkV05OCKE1yIBAZfHpmAnqATRlRrXBnDTn8KFWtliFKKO0fZHiKEPRzxzMB3CKvDSgquxoePtSyVtPguaR1UNYrK3h"}],"group":"cf-nel","max_age":604800}', 'NEL': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'Server': 'cloudflare', 'CF-RAY': '822369c7eefbd00d-SJC'})
HTTP response body: {"code":401,"message":"Invalid API key, please visit https://e2b.dev/docs?reason=sdk-missing-api-key to get your API key."}


In [ ]:
agent = OpenAIAssistantRunnable(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "function", "function": function}],
    model="gpt-4-1106-preview",
    as_agent=True
)
agent.create()

#### Using AgentExecutor


In [8]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "What's 10 - 4 raised to the 2.7"})

ValueError: {'id': 'run_08of98E8mkOkvvLevGlmxTpF', 'assistant_id': 'asst_8dAlzPI0zvjOaaenvwNCtIEl', 'cancelled_at': None, 'completed_at': None, 'created_at': 1699337429, 'expires_at': None, 'failed_at': 1699337439, 'file_ids': [], 'instructions': 'You are a personal math tutor. Write and run code to answer math questions.', 'last_error': {'code': 'rate_limit_exceeded', 'message': "We're currently processing too many requests - please try again later."}, 'metadata': {}, 'model': 'gpt-4-1106-preview', 'object': 'thread.run', 'required_action': None, 'started_at': 1699337429, 'status': 'failed', 'thread_id': 'thread_wcdfruB9FPwjs27TbBv0p45U', 'tools': [{'function': {'description': 'Evaluates python code in a sandbox environment. The environment is long running and exists across multiple executions. You must send the whole script every time and print your outputs. Script should be pure python code that can be evaluated. It should be in python format NOT markdown. The code should NOT be wra

#### Custom execution

In [ ]:
agent = OpenAIAssistantRunnable(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "function", "function": function}],
    model="gpt-4-1106-preview",
    as_agent=True
)
agent.create()

In [5]:
from langchain.schema.agent import AgentFinish

tool_map = {tool.name: tool for tool in tools}
response = agent.invoke({"content": "What's 10 - 4 raised to the 2.7"})
while not isinstance(response, AgentFinish):
    tool_outputs = []
    for action in response:
        tool_output = tool_map[action.tool].invoke(action.tool_input)
        print(action.tool, action.tool_input, tool_output)
        tool_outputs.append({"output": tool_output, "tool_call_id": action.tool_call_id})
    response = agent.invoke({"tool_outputs": tool_outputs, "run_id": action.run_id})
    
print(response.return_values["output"])

ValueError: {'id': 'run_6QY8OJeyV2M4BjUQMdWkAn1i', 'assistant_id': 'asst_8dAlzPI0zvjOaaenvwNCtIEl', 'cancelled_at': None, 'completed_at': None, 'created_at': 1699331591, 'expires_at': None, 'failed_at': 1699331601, 'file_ids': [], 'instructions': 'You are a personal math tutor. Write and run code to answer math questions.', 'last_error': {'code': 'rate_limit_exceeded', 'message': "We're currently processing too many requests - please try again later."}, 'metadata': {}, 'model': 'gpt-4-1106-preview', 'object': 'thread.run', 'required_action': None, 'started_at': 1699331591, 'status': 'failed', 'thread_id': 'thread_wcdfruB9FPwjs27TbBv0p45U', 'tools': [{'function': {'description': 'Evaluates python code in a sandbox environment. The environment is long running and exists across multiple executions. You must send the whole script every time and print your outputs. Script should be pure python code that can be evaluated. It should be in python format NOT markdown. The code should NOT be wra

## [JSON mode](https://platform.openai.com/docs/guides/text-generation/json-mode)

Constrain the model to only generate valid JSON. Note that you must include a system message with instructions to use JSON for this mode to work.

Only works with certain models. 

In [8]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(
    response_format={"type": "json_object"}
)

output = chat.invoke(
    [
        SystemMessage(
            content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
        ),
        HumanMessage(
            content="Google was founded in the USA, while Deepmind was founded in the UK"
        ),
    ]
)
print(output.content)

{
  "companies": [
    {
      "name": "Google",
      "origin": "USA"
    },
    {
      "name": "Deepmind",
      "origin": "UK"
    }
  ]
}


In [9]:
import json

json.loads(output.content)

{'companies': [{'name': 'Google', 'origin': 'USA'},
  {'name': 'Deepmind', 'origin': 'UK'}]}

## [System fingerprint](https://platform.openai.com/docs/guides/text-generation/reproducible-outputs)

OpenAI sometimes changes model configurations in a way that impacts outputs. Whenever this happens, the system_fingerprint associated with a generation will change.

In [2]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106")
output = chat.generate(
    [
        [
            SystemMessage(
                content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
            ),
            HumanMessage(
                content="Google was founded in the USA, while Deepmind was founded in the UK"
            ),
        ]
    ]
)
print(output.llm_output)

{'token_usage': {'completion_tokens': 43, 'prompt_tokens': 49, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_eeff13170a'}
